# 外国為替レートと経済指標との相関

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

## 為替レートデータの読み込み

In [2]:
df_exrate = pd.read_excel('Foreign_JPY_18_23.xlsx', sheet_name='RawData')
df_exrate.head()

,date,USD,GBP,EUR,CAD,AUD,NZD
0,2018-01-04,112.75,152.30,135.35,89.83,88.22,79.88
1,2018-01-05,112.84,153.06,136.21,90.36,88.62,80.75
2,2018-01-09,113.15,153.50,135.43,91.15,88.82,81.23
3,2018-01-10,112.45,152.17,134.21,90.17,87.87,80.39
4,2018-01-11,111.52,150.64,133.30,88.93,87.83,80.32


## 原油価格データの読み込み

In [3]:
df_temp_1 = pd.read_excel('oil_price_18_23.xlsx', sheet_name='RawData')
df_temp_1.head()

,Date,Price,Open,High,Low
0,04/04/2023,80.81,80.42,80.91,80.41
1,04/03/2023,80.42,80.10,81.69,79.00
2,04/02/2023,80.51,80.27,81.58,80.22
3,03/31/2023,75.67,74.37,75.72,73.77
4,03/30/2023,74.37,72.98,74.63,72.61


In [4]:
# 日付文字列を日付値に変換する
df_petro = df_temp_1.assign(tdatetime = df_temp_1['Date'].apply(lambda s: dt.strptime(s, '%m/%d/%Y'))).\
    drop(columns='Date').rename(columns={'tdatetime':'date'})
df_petro.head()

,Price,Open,High,Low,date
0,80.81,80.42,80.91,80.41,2023-04-04
1,80.42,80.10,81.69,79.00,2023-04-03
2,80.51,80.27,81.58,80.22,2023-04-02
3,75.67,74.37,75.72,73.77,2023-03-31
4,74.37,72.98,74.63,72.61,2023-03-30


## 石炭価格データの読み込み

In [5]:
df_coal = pd.read_excel('coal_price_18_23.xlsx', sheet_name='RawData').rename(columns={'Date':'date'})
df_coal.head()

,date,Price,Open,High,Low
0,2023-04-03,200.00,197.00,198.00,197.00
1,2023-03-31,177.25,177.25,177.25,177.25
2,2023-03-30,176.75,176.75,176.75,176.75
3,2023-03-29,176.60,176.60,176.60,176.60
4,2023-03-28,176.50,176.50,176.50,176.50
